In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
from Extractors import *
from rdflib import Graph, RDF, URIRef, Literal
from rdflib.namespace import RDF, XSD, FOAF, Namespace

In [12]:
graphe = Graph()
ace = Namespace("http://www.semanticweb.org/acemap#")
auteur1 = URIRef("http://www.semanticweb.org/acemap#001")
auteur2 = URIRef("http://www.semanticweb.org/acemap#002")
auteur3 = URIRef("http://www.semanticweb.org/acemap#003")
paper1 = URIRef("http://www.semanticweb.org/acemap#010")
paper2 = URIRef("http://www.semanticweb.org/acemap#020")
paper3 = URIRef("http://www.semanticweb.org/acemap#030")
field1 = URIRef("http://www.semanticweb.org/acemap#100")
field2 = URIRef("http://www.semanticweb.org/acemap#200")
field3 = URIRef("http://www.semanticweb.org/acemap#300")
graphe.add((auteur1, RDF.type, ace.Author))
graphe.add((auteur2, RDF.type, ace.Author))
graphe.add((auteur3, RDF.type, ace.Author))
graphe.add((paper1, RDF.type, ace.Paper))
graphe.add((paper2, RDF.type, ace.Paper))
graphe.add((paper3, RDF.type, ace.Paper))
graphe.add((field1, RDF.type, ace.Field))
graphe.add((field2, RDF.type, ace.Field))
graphe.add((field3, RDF.type, ace.Field))
graphe.add((auteur1, ace.author_name, Literal("Remi DURET", datatype=XSD.string)))
graphe.add((auteur2, ace.author_name, Literal("Waris RADJI", datatype=XSD.string)))
graphe.add((auteur3, ace.author_name, Literal("Colin ESPINAS", datatype=XSD.string)))
graphe.add((field1, ace.field_name, Literal("Genie Logiciel", datatype=XSD.string)))
graphe.add((field2, ace.field_name, Literal("Data Science", datatype=XSD.string)))
graphe.add((field3, ace.field_name, Literal("Web development", datatype=XSD.string)))
graphe.add((paper1, ace.paper_title, Literal("Optisimation des applications de bureau", datatype=XSD.string)))
graphe.add((paper2, ace.paper_title, Literal("Les algorithmes de data mining", datatype=XSD.string)))
graphe.add((paper3, ace.paper_title, Literal("Faire un framework complet", datatype=XSD.string)))
graphe.add((auteur1, ace.author_is_in_field, field1))
graphe.add((auteur2, ace.author_is_in_field, field1))
graphe.add((auteur2, ace.author_is_in_field, field2))
graphe.add((auteur3, ace.author_is_in_field, field1))
graphe.add((auteur3, ace.author_is_in_field, field2))
graphe.add((paper1, ace.paper_is_in_field, field1))
graphe.add((paper2, ace.paper_is_in_field, field1))
graphe.add((paper2, ace.paper_is_in_field, field2))
graphe.add((paper3, ace.paper_is_in_field, field1))
graphe.add((paper3, ace.paper_is_in_field, field2))
graphe.add((paper1, ace.paper_is_written_by, auteur1))
graphe.add((paper2, ace.paper_is_written_by, auteur2))
graphe.add((paper2, ace.paper_is_written_by, auteur3))
graphe.add((paper3, ace.paper_is_written_by, auteur3))
graphe.add((paper2, ace.paper_cit_paper, paper1))
graphe.add((paper3, ace.paper_cit_paper, paper1))
graphe.add((paper3, ace.paper_cit_paper, paper2))
graphe.add((paper1, ace.paper_publish_date, Literal("2015-03-26", datatype=XSD.date)))
graphe.add((paper2, ace.paper_publish_date, Literal("2018-05-12", datatype=XSD.date)))
graphe.add((paper3, ace.paper_publish_date, Literal("2020-02-07", datatype=XSD.date)))

In [34]:
def AjouterPrefixe(prefixe, chaine) :
    """
    Fonction qui ajouter un préfixe à une chaine de caractère et remplace les " " par des "_"
    retourne une chaine de caractères avec le préfixe placé devant
    exemple : AjouterPrefixe("AUTH", "Albert Einstein") retourne "AUTH_Albert_Einstein"
    """
    pref = prefixe + "_"
    new_chaine = pref + chaine.replace(" ", "_")
    return new_chaine

def YearFromDate(date) :
    """
    Extrait l'année d'un date qui est au format xsd.date (YYYY-MM-DD)
    Retourne la chaine de caractère correspondant à l'année de la date
    """
    return date[:4]  


def ExtraireAuteurs(graphe) :
    """
    Extrait tous les id d'auteurs dans un tableau
    prends un paramètre un graphe de rdflib
    retourne un tableau avec tous les id des auteurs
    """
    auteurs = []
    authors = graphe.subjects(RDF.type, ace.Author)
    for author in authors :
        auteurs.append(author)
    return auteurs


def ExtrairePublications(graphe) :
    """
    Extrait tous les id de publication dans un tableau
    prends un paramètre un graphe de rdflib
    retourne un tableau avec tous les id des publications
    """
    publi = []
    papers = graphe.subjects(RDF.type, ace.Paper)
    for paper in papers :
        publi.append(paper)
    return publi


def ExtraireConceptes(graphe) :
    """
    Extrait tous les id de thématiques dans un talbeau
    prend en paramètres un graphe de rdflib
    retourne un tableau avec tous les id des thématiques/field/domaines/concepts (CONC)
    """
    them = []
    fields = graphe.subjects(RDF.type, ace.Field)
    for field in fields : 
        them.append(field)
    return them


def ExtraireNomAuteurs(graphe, auteurs) :
    """
    Extrait les identifiants des auteurs et leurs noms présents dans acemap
    prend en paramètre un objet graphe de la RDFlib
    retourne une dictionnaire qui associe authorID à son nom
    """

    IDToAuthors = dict()
    for auteur in auteurs :
        noms = graphe.objects(auteur, ace.author_name)
        for nom in noms :
            IDToAuthors[auteur] = nom
    return IDToAuthors


def ExtraireNomConceptes(graphe, conceptes) :
    """
    Extrait les nom des différents conceptes/domaines/field et les associes à leurs ID
    prend en parametre un gaphe rdflib et la liste des id des conceptes
    retourne un dictionnaire associant un fieldid a son nom
    """
    IDToField = dict()
    for concepte in conceptes :
        noms = graphe.objects(concepte, ace.field_name)
        for nom in noms : 
            IDToField[concepte] = nom
    return IDToField


def PaperToYear(graphe, publications) :
    """
    Extrait les dates et les associ a leurs publication
    prend en paramètre un graphe rdflib et la liste des publication
    retourne un dicitonnaire associant publicationID à sa date de publication (jespere qu'il en a qu'une seul)
    exemple : { paperID : date, paperID : date ....... }
    """
    paperToYear = dict()
    for publication in publications :
        dates = graphe.objects(publication, ace.paper_publish_date)
        for date in dates :
            paperToYear[publication] = date
    return paperToYear


def PaperToField(graphe, publications) :
    """
    Extrait les domaines des publications
    prend en paramètre un objet graphe de rdflib et un liste d'id de publication sous forme d'URI
    retourne un dictionnaire associant à un paperID tous les FieldID auxquelles il est rataché
    { paperID : [fieldID, fieldID, fieldID], paperID : [fieldID, ...] ... }
    """
    paperToField = dict()
    for publication in publications :
        domaines = graphe.objects(publication, ace.paper_is_in_field)
        paperToField[publication] = []
        for domaine in domaines :
            paperToField[publication].append(domaine)
    return paperToField


def AuthorToField(graphe, auteurs):
    """
    Extrait les domaines des auteurs
    prend en paramètre un objet graphe de rdflib et un liste d'id d'autheurs sous forme d'URI
    retourne un dictionnaire associant à un authorID tous les FieldID auxquelles il est rataché
    { authorID : [fieldID, fieldID, fieldID], authorID : [fieldID, ...] ... }
    """
    authIDToFieldID = dict()
    for auteur in auteurs :
        domaines = graphe.objects(auteur, ace.author_is_in_field)
        authIDToFieldID[auteur] = []
        for domaine in domaines :
            authIDToFieldID[auteur].append(domaine)
    return authIDToFieldID


def AuthorWritePaper(graphe, auteurs):
    """
    Extrait les papiers qu'un auteur a écrit
    prend en paramètre la liste des auteurs et le graphe de rdflib
    retourne un dictionnaire associant un auteur a toutes ces publication
    { authorID : [paperID, paperID, paperID], authorID : [paperID, ...] ... }
    """
    authWritePaper = dict()
    for auteur in auteurs :
        authWritePaper[auteur] = []
        papers = graphe.subjects(ace.paper_is_written_by, auteur) 
        for paper in papers :
            authWritePaper[auteur].append(paper)
    return authWritePaper

def ExtrairesDate(graphe, authorWritePaper, auteurs):
    """
    Extrait les dates de publication d'un auteur.
    prend en paramètre un graphe rdf une liste d'authorID sous forme d'URI et un dictionnaire qui associe un auteurs a ces publication
    retourne un dictionnaire associant des authorID à des dates de publication 
    { authorID : [date, date, date], authorID : [date, ...] ... }
    """
    authIDToYears = dict()
    for auteur in auteurs :
        authIDToYears[auteur] = []
        #on va générer la liste de toutes les publication qui ont été ecrite par l'auteur en question
        for publication in authorWritePaper[auteur] :
            dates = graphe.objects(publication, ace.paper_publish_date)
            for date in dates :
                authIDToYears[auteur].append(date)
    return authIDToYears

def ExtraireAuteursPubli(graphe, publications) :
    """
    Extrait les auteurs pour toutes les publication
    prend en parametre un graphe rdf
    retourne un dictionnaire qui associe les paperID a tous ses authorID
    { paperID : [authorID, authoID, authorID], paperID : [authorID, ...] ... }
    """
    paperIDToAuthorID = dict()
    for publication in publications :
        auteurs = graphe.objects(publication, ace.paper_is_written_by)
        paperIDToAuthorID[publication] = []
        for auteur in auteurs :
            paperIDToAuthorID[publication].append(auteur)
    return paperIDToAuthorID


def ExtrairePubliCit(graphe, publications) :
    """
    Extrait les publication cités par des publciation
    prend en paramètre la liste des publication
    retourne un dictionnaire associé un publication a toutes les publication qu'elle cite
    { paperID : [paperID, paperID ...] , paperID : ......}
    """
    paperCit = dict()
    for publication in publications :
        paperCit[publication] = []
        cites = graphe.objects(publication, ace.paper_cit_paper)
        for cite in cites :
            paperCit[publication].append(cite)
    return paperCit

def AuthToPaperCit(paperCitPaper, paperToAuthor, publications, auteurs) :
    """
    Crée un dictionnaire associant un auteur au publication qu'il a cité
    prend en paramètres le dictionnaire avec les publication et leur citation, le dictionnaire des publication et de leurs auteurs, le tableau des publications
    renvoit un dictionnaire des auteurs et de leur citation
    { authorID : [paperID, paperID, ...] , ... }
    """
    authToPaperCit = dict()
    for auteur in auteurs :
        authToPaperCit[auteur] = []
    for publication in publications :
        paperCits = paperCitPaper[publication]
        for paperCit in paperCits :
            authors = paperToAuthor[publication]
            for author in authors :
                if not (paperCit in authToPaperCit[author]) :
                    authToPaperCit[author].append(paperCit)
    return authToPaperCit


def Coauteurs(paperToAuthor, auteurs, authorToPaper):
    """
    Crée  un dictionnaire associant chaque auteur à ses coauteurs
    prend en paramètre un dictionnaire associant les publication à leurs auteurs (cf ExtraireAuteursPubli), un tableau des auteurs et un dictionnaire associant les auteurs a leurs publication
    retourne un dictionnaire sous la forme { authID : [authID, authID ...] , authID : ......}
    """
    coaut = dict()
    for auteur in auteurs :
        coaut[auteur] = []
    for auteur in auteurs :
        for publication in authorToPaper[auteur] :
            for aut in paperToAuthor[publication] :
                if (aut != auteur) and (not (aut in coaut[auteur])) :
                    coaut[auteur].append(aut)
                    if( not auteur in coaut[aut]) :
                        coaut[aut].append(auteur)
    return coaut

def Citation(authorCitPaper, auteurs, paperToAuthor) :
    """
    Crée un dictionnaire associant chaque auteur aux auteurs qu'il a cité
    prend en parametres un dictionnaire associant un auteur aux publication qu'il a cité, un talbeau d'auteurs, un dictionnaire associant des publications à leurs auteurs
    retourne un dictionnaire associant un auteur aux auteurs qu'il a cité
    exemple : { authID : [authID, authID ...] , authID : ......}
    """
    citAuteur = dict()
    for auteur in auteurs :
        citAuteur[auteur] = []
        for publiCit in authorCitPaper[auteur] :
            for authorCit in paperToAuthor[publiCit] :
                if not authorCit in citAuteur[auteur] :
                    citAuteur[auteur].append(authorCit)
    return citAuteur

In [37]:
auteurs = ExtraireAuteurs(graphe)
publications = ExtrairePublications(graphe)
domaines = ExtraireConceptes(graphe)
idToAuth = ExtraireNomAuteurs(graphe, auteurs)
idToField = ExtraireNomConceptes(graphe, domaines)
paperToDate = PaperToYear(graphe, publications)
paperToField = PaperToField(graphe, publications)
authorToField = AuthorToField(graphe, auteurs)
authorToPaper = AuthorWritePaper(graphe, auteurs)
authorToDate = ExtrairesDate(graphe, authorToPaper, auteurs)
paperToAuthor = ExtraireAuteursPubli(graphe, publications)
paperCitPaper = ExtrairePubliCit(graphe, publications)
authToPaperCit = AuthToPaperCit(paperCitPaper, paperToAuthor, publications, auteurs)
coauteurs = Coauteurs(paperToAuthor, auteurs, authorToPaper)
citations = Citation(authToPaperCit, auteurs, paperToAuthor) 
print(citations)

{rdflib.term.URIRef('http://www.semanticweb.org/acemap#001'): [], rdflib.term.URIRef('http://www.semanticweb.org/acemap#002'): [rdflib.term.URIRef('http://www.semanticweb.org/acemap#001')], rdflib.term.URIRef('http://www.semanticweb.org/acemap#003'): [rdflib.term.URIRef('http://www.semanticweb.org/acemap#002'), rdflib.term.URIRef('http://www.semanticweb.org/acemap#003'), rdflib.term.URIRef('http://www.semanticweb.org/acemap#001')]}
